## Tutorial (1): RAG + Llama2
### This notebook is adapted from [llama-recipes](https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/HelloLlamaLocal.ipynb) and shows:
* How to run Llama2 locally on a Linux using llama-cpp-python and the llama-cpp's quantized Llama2 model
* How to use LangChain to ask Llama general questions
* How to use LangChain to load a recent PDF doc - lettre connectées IMT in EN pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. 

We start by installing necessary requirements and import packages we will be using in this example.
- [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) a simple Python bindings for [llama.cpp](https://github.com/ggerganov/llama.cpp) library
- pypdf gives us the ability to work with pdfs
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

In [ ]:
# install all the required packages for the demo
#!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python #for MAC
!CMAKE_ARGS="-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS" pip install llama-cpp-python # for LINUX
!pip install pypdf sentence-transformers chromadb langchain

In [2]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [3]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model. 

Specify model path with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing), or to the `ggml-model-q4_0.gguf` file built with the following commands:
```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0
```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [4]:

llm = LlamaCpp(
    model_path="/home/g20lioi/Downloads/ggml-model-q4_0.gguf",
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager, 
    verbose=True,
)

llama_model_loader: loaded meta data with 16 key-value pairs and 363 tensors from /home/g20lioi/Downloads/ggml-model-q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

With the model set up, you are now ready to ask some questions. 

Here is an example of the simplest way to ask the model some general questions.

In [5]:
question = "who wrote the book Germinal?"
answer = llm(question)

/home/g20lioi/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




The book "Germinal" was written by Émile Zola. It was published in 1885 and is considered one of Zola's most important works, as well as a classic of French literature. The novel is set in a coal mining region of France and explores themes such as class struggle, the exploitation of workers, and the struggle for social justice.


llama_print_timings:        load time =    1778.91 ms
llama_print_timings:      sample time =      48.97 ms /    84 runs   (    0.58 ms per token,  1715.37 tokens per second)
llama_print_timings: prompt eval time =    1778.83 ms /     8 tokens (  222.35 ms per token,     4.50 tokens per second)
llama_print_timings:        eval time =   35688.58 ms /    83 runs   (  429.98 ms per token,     2.33 tokens per second)
llama_print_timings:       total time =   37822.78 ms


Alternatively, you can use LangChain's `PromptTemplate` for some flexibility in your prompts and questions.

For more information on LangChain's prompt template visit this [link](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

Now, let's see how Llama2 hallucinates, because it did not have knowledge about Llama2 at the time it was trained. 
By default it behaves like a know-it-all expert who will not say "I don't know".

In [6]:
question= "What's the Ocean Hackaton?"
answer = llm(question)

Llama.generate: prefix-match hit




The Ocean Hackathon is a global online hackathon that brings together innovators, entrepreneurs, and experts to develop solutions for sustainable ocean development. The event is organized by the United Nations Development Programme (UNDP) and partners, and it aims to find new and innovative ways to address the challenges facing the world's oceans.

The hackathon focuses on three main themes:

1. Sustainable Fisheries and Aquaculture
2. Ocean Conservation and Restoration
3. Blue Economy and Innovation

Participants can form teams and submit their ideas, prototypes, or solutions to address these challenges. The event provides a platform for participants to collaborate, share knowledge, and gain exposure to potential investors, mentors, and industry experts.

The Ocean Hackathon is open to anyone with an interest in ocean conservation and sustainable development. Participants can come from diverse backgrounds, including technology, design, science, business, and social entrepreneurship.


llama_print_timings:        load time =    1778.91 ms
llama_print_timings:      sample time =     185.84 ms /   256 runs   (    0.73 ms per token,  1377.55 tokens per second)
llama_print_timings: prompt eval time =    2078.46 ms /     8 tokens (  259.81 ms per token,     3.85 tokens per second)
llama_print_timings:        eval time =  140115.35 ms /   256 runs   (  547.33 ms per token,     1.83 tokens per second)
llama_print_timings:       total time =  143561.50 ms


One way we can fix the hallucinations is to use RAG, to augment it with more recent or custom data that holds the information for it to answer correctly.

First we load the english version of *IMT lettre connectées* using LangChain's [PDF loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [7]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/home/g20lioi/Downloads/connectees_12_23_IMT.pdf")
documents = loader.load()

In [8]:
# quick check on the loaded document for the correct pages etc
print(len(documents), documents[0].page_content[0:300])

22 Connected.es Newsletter #56
22/12
 In bold, the subjects of most direct interest to students
Contents
•Director’s editorial 
•General information   
•Organisation of the school: Creation of two departments, Appointments 
•Results of the student representative elections   
•2024 greetings card 
•Grad


Next we will store our documents. 
There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

We will also import the `HuggingFaceEmbeddings` and `RecursiveCharacterTextSplitter` to assist in storing the documents.

In [9]:
from langchain.vectorstores import Chroma

# embeddings are numerical representations of the question and answer text
from langchain.embeddings import HuggingFaceEmbeddings

# use a common text splitter to split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 


In [10]:
# split the loaded documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

# create the vector db to store all the split chunks as embeddings
embeddings = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(
    documents=all_splits,
    embedding=embeddings,
)


We then use ` RetrievalQA` to retrieve the documents from the vector database and give the model more context on Llama 2, thereby increasing its knowledge.

In [ ]:
# use another LangChain's chain, RetrievalQA, to associate Llama with the loaded documents stored in the vector db
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

For each question, LangChain performs a semantic similarity search of it in the vector db, then passes the search results as the context to the model to answer the question.

It takes close to 2 minutes to return the result (but using other vector stores other than Chroma such as FAISS can take longer) because Llama2 is running on a local machine. 
To get much faster results, you can use a cloud service with GPU used for inference - see HelloLlamaCloud for a demo.

In [ ]:
question = "What's the Ocean Hackaton?"
result = qa_chain({"query": question})